In [163]:
import numpy as np
import scipy as sc
import sympy as sp
np.set_printoptions(precision=3, suppress=True)

import cdd
import jax
import jax.numpy as jp
from jax.config import config
config.update("jax_enable_x64", True)

# Examples

In [164]:
# Example 1

states = np.array([[1,0,1],\
                   [1,0,-1],\
                   [1,1,0],\
                   [1,-1,0]]).T/2
effects = np.array([[1,0,1],
                    [1,0,-1],\
                    [1,1,0],\
                    [1,-1,0]]).T/2
unit_effect = np.array([1,0,0])
maximally_mixed_state = np.array([1/2,0,0])

In [106]:
# Example 2

states = np.array([[1,0,0,0],\
                   [0,1,0,0],\
                   [0,0,1,0],\
                   [0,0,0,1]])
effects = np.array([[1,1,0,0],
                    [0,1,1,0],\
                    [0,0,1,1],\
                    [1,0,0,1]])
unit_effect = np.array([1,1,1,1])
maximally_mixed_state = np.mean(states,axis=1)

In [156]:
# Boxworld

states = np.array([[1,1,0],\
                   [1,0,1],\
                   [1,-1,0],\
                   [1,0,-1]]).T
effects = np.array([[1,-1,-1],\
                    [1,1,-1],\
                    [1,1,1],\
                    [1,-1,1]]).T/2
unit_effect = np.array([1,0,0])
maximally_mixed_state = np.array([1,0,0])

# DiscoverEmbedding

In [157]:
def convex_hull_rays(M):
    C = cdd.Matrix(M)
    C.rep_type = cdd.RepType.INEQUALITY
    return np.array(cdd.Polyhedron(C).get_generators())

def construct_accessible_fragment(M):
    I = np.array(sp.Matrix(M).rref()[0], dtype=float)
    I = np.unique(I, axis=0)[::-1]
    inclusion_map = I[~np.all(np.isclose(I,0), axis=1)].T
    projection_map = np.linalg.pinv(inclusion_map.T)
    accessible_fragment = M @ projection_map
    return projection_map, inclusion_map, accessible_fragment

In [158]:
print("Stage 1: Constructing an accessible GPT fragment.\n")

P_states, I_states, A_states = construct_accessible_fragment(states.T)
P_effects, I_effects, A_effects = construct_accessible_fragment(effects.T)
A_unit_effect = P_effects.T @ unit_effect
depolarizing_map = np.outer(unit_effect, maximally_mixed_state)

print("Proj_Ω: \n%s\n" % P_states)
print("Proj_ξ: \n%s\n" % P_effects)
print("Inc_Ω: \n%s\n" % I_states)
print("Inc_ξ: \n%s\n" % I_effects)
print("V_Ω^A: \n%s\n" % A_states)
print("V_ξ^A: \n%s\n" % A_effects)
print("|u_a>: \n%s\n" % A_unit_effect)
print("DepolarizingMap: \n%s" % depolarizing_map)

Stage 1: Constructing an accessible GPT fragment.

Proj_Ω: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Proj_ξ: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Inc_Ω: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Inc_ξ: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

V_Ω^A: 
[[ 1.  1.  0.]
 [ 1.  0.  1.]
 [ 1. -1.  0.]
 [ 1.  0. -1.]]

V_ξ^A: 
[[ 0.5 -0.5 -0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0.5]
 [ 0.5 -0.5  0.5]]

|u_a>: 
[1. 0. 0.]

DepolarizingMap: 
[[1 0 0]
 [0 0 0]
 [0 0 0]]


In [159]:
print("Stage 2: Finding the facets of the GPT state and effect hypercones.\n")

H_states = convex_hull_rays(A_states)
H_effects = convex_hull_rays(A_effects)

print("H_Ω: \n%s\n" % H_states)
print("H_ξ: \n%s" % H_effects)

Stage 2: Finding the facets of the GPT state and effect hypercones.

H_Ω: 
[[ 1.  1. -1.]
 [ 1.  1.  1.]
 [ 1. -1.  1.]
 [ 1. -1. -1.]]

H_ξ: 
[[ 1.  0.  1.]
 [ 1.  1.  0.]
 [ 1.  0. -1.]
 [ 1. -1.  0.]]


In [160]:
print("Stage 3: Finding the Robustness of Nonembeddability to Depolarizing Noise.\n")

n = H_states.shape[0]
V = np.random.randn(n**2 + 1)

@jax.jit
def linear_program(V):
    r = V[0]
    sigma = V[1:].reshape(n,n)
    return jp.linalg.norm(r*(I_effects.T @ depolarizing_map @ I_states) + (1-r)*(I_effects.T @ I_states) - \
                              H_effects.T @ sigma @ H_states) + r

@jax.jit
def positivity(V):
    return V

result = sc.optimize.minimize(linear_program, V, jac=jax.jit(jax.jacrev(linear_program)),\
                                  constraints=[{"type": "ineq",\
                                                "fun": positivity,\
                                                "jac": jax.jit(jax.jacrev(positivity))}],\
                                  method="SLSQP")
r = result.x[0]
sigma = result.x[1:].reshape(n,n)

print("σ: \n%s\n" % sigma)
print("r: \n%s" % np.round(r, 3))

if not np.isclose(r, 0):
    B = r*(I_effects.T @ depolarizing_map @ I_states) + (1-r)*(I_effects.T @ I_states)
    print("\nB_embeddable^A: \n%s\n" % B)

Stage 3: Finding the Robustness of Nonembeddability to Depolarizing Noise.

σ: 
[[-0.     0.125  0.125 -0.   ]
 [ 0.125  0.125 -0.    -0.   ]
 [ 0.125 -0.    -0.     0.125]
 [-0.     0.     0.125  0.125]]

r: 
0.5

B_embeddable^A: 
[[1.  0.  0. ]
 [0.  0.5 0. ]
 [0.  0.  0.5]]



In [161]:
print("Stage 4: Constructing a Simplex Embedding from a Simplicial-Cone Embedding.\n")

simplicial_cone_embedding_states = H_states
simplicial_cone_embedding_effects = sigma.T @ H_effects

simplex_embedding_states = \
    np.array([simplicial_cone_embedding_states[i]*(simplicial_cone_embedding_effects[i] @ A_unit_effect)\
         for i in range(len(simplicial_cone_embedding_states))])

simplex_embedding_effects = \
    np.array([simplicial_cone_embedding_effects[i]/(simplicial_cone_embedding_effects[i] @ A_unit_effect)\
         for i in range(len(simplicial_cone_embedding_effects))\
              if simplicial_cone_embedding_effects[i] @ A_unit_effect > 0])

print("τ_Ω^A: \n%s\n" % simplex_embedding_states)
print("τ_ξ^A: \n%s\n" % simplex_embedding_effects)

ontic_states = np.array([simplex_embedding_states @ state for state in A_states]).T; ontic_states
response_functions = np.array([simplex_embedding_effects @ effect for effect in A_effects]).T; response_functions

print("DepolarizedNCModel_Ω: \n%s\n" % ontic_states)
print("DepolarizedNCModel_ξ: \n%s" % response_functions)

Stage 4: Constructing a Simplex Embedding from a Simplicial-Cone Embedding.

τ_Ω^A: 
[[ 0.25  0.25 -0.25]
 [ 0.25  0.25  0.25]
 [ 0.25 -0.25  0.25]
 [ 0.25 -0.25 -0.25]]

τ_ξ^A: 
[[ 1.   0.5 -0.5]
 [ 1.   0.5  0.5]
 [ 1.  -0.5  0.5]
 [ 1.  -0.5 -0.5]]

DepolarizedNCModel_Ω: 
[[0.5 0.  0.  0.5]
 [0.5 0.5 0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  0.5 0.5]]

DepolarizedNCModel_ξ: 
[[ 0.5  1.   0.5 -0. ]
 [ 0.   0.5  1.   0.5]
 [ 0.5 -0.   0.5  1. ]
 [ 1.   0.5 -0.   0.5]]


In [162]:
print("Compare:\n")
print("R.T @ O: \n%s\n" % (response_functions.T @ ontic_states))
print("E.T @ S: \n%s\n" % (effects.T @ states))

Compare:

R.T @ O: 
[[0.25 0.25 0.75 0.75]
 [0.75 0.25 0.25 0.75]
 [0.75 0.75 0.25 0.25]
 [0.25 0.75 0.75 0.25]]

E.T @ S: 
[[0. 0. 1. 1.]
 [1. 0. 0. 1.]
 [1. 1. 0. 0.]
 [0. 1. 1. 0.]]

